In [1]:
import itertools
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.notebook import trange, tqdm, tqdm_notebook
from joblib import Parallel, delayed
import re
import h5py
import tifffile as tiff


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()
data_processed = data_dir / 'processed'
data_raw = r'Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\Protein baseline vs PLA'


# Get info

In [49]:
markers_map = {
    '2': {
        1: 'DNA', 
        3: 'Cyclin D1',
    },
    '3': {
        1: 'DNA', 
        3: 'CDK4',
    },
    '4': {
        1: 'DNA', 
        3: 'SOX2',
    },
    '5': {
        1: 'DNA', 
        3: 'Oct4',
    },
    '6': {
        1: 'DNA', 
        3: 'CycD1/CDK4',
    },
    '7': {
        1: 'DNA', 
        3: 'Sox2/Oct4',
    },
    '8': {
        1: 'DNA', 
        3: 'Wnt1/Tead',
    },
    'cycle2': {
        1: 'DNA', 
        2: 'Concanavalin A',
        3: 'Phalloidin',
        4: 'WGA'
    },
    'cycle2_sec': {
        1: 'DNA', 
        3: 'Wnt1',
        4: 'Tead'
    }
}

def get_info(data_raw, marker_dict):
    conditions = []
    fovs = []
    cycles = []
    channels = []
    markers = []
    paths = [] 
    
    # Loop through image folder
    for (dirpath, dirnames, filenames) in os.walk(data_raw):
        if 'Dont use' in dirpath:
            continue
            
        for name in sorted(filenames):
            if "tif" in name and "sti" in name \
            and 'overlay' not in name \
            and 'Composite' not in name:
                # Get information from image name
                condition = 'Control' if filenames[-1][1] == 'B' else 'Treated'
                well = filenames[-1][-1]
                n_split = name.split('_')
                
                ch = int(n_split[-1][-5])
                if 'Seg' in dirpath:
                    cycle = 'cycle2'
                    try: marker = marker_dict[cycle][ch] 
                    except: continue
                    cycle = '2'
                elif 'Second' in dirpath:
                    cycle = 'cycle2_sec'
                    try: marker = marker_dict[cycle][ch] 
                    except: continue
                    cycle = '2'
                else:
                    cycle = well
                    try: marker = marker_dict[cycle][ch] 
                    except: continue
                    cycle = '1'
        
                conditions.append(condition)
                fovs.append(well)
                cycles.append(cycle)
                channels.append(ch)
                markers.append(marker)
                paths.append(os.path.join(dirpath, name))
                
    info = {
            "Condition": conditions,
            "FOV": fovs,
            "Cycle": cycles,
            "Channels": channels,
            "Markers": markers,
            "Path": paths
        }

    df = pd.DataFrame(info)
    return df

In [50]:
df_meta_path = data_dir / 'basline' / 'metadata' / 'info.csv'
df_meta_path.parent.mkdir(parents=True, exist_ok=True)


df_exist = df_meta_path.is_file()

if not df_exist:
    print('Created df')
    df = get_info(data_raw, markers_map)
    df.to_csv(df_meta_path, index=False)
else:
    print('Loaded df')
    df = pd.read_csv(df_meta_path)

Created df


In [51]:
# # Loop through image folder
# for (dirpath, dirnames, filenames) in os.walk(data_raw):
#     for name in sorted(filenames):
#         if "tif" in name and "sti" in name \
#         and 'overlay' not in name \
#         and 'Composite' not in name:
#             print(name, filenames[-1])
#             break

In [52]:
df

,Condition,FOV,Cycle,Channels,Markers,Path
0,Control,2,1,1,DNA,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."
1,Control,2,1,3,Cyclin D1,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."
2,Control,3,1,1,DNA,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."
3,Control,3,1,3,CDK4,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."
4,Control,4,1,1,DNA,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."
...,...,...,...,...,...,...
77,Control,8,2,3,Wnt1,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."
78,Control,8,2,4,Tead,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."
79,Treated,8,2,1,DNA,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."
80,Treated,8,2,3,Wnt1,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."


# Save HDF5

In [53]:
import h5py

def save_hdf5(
    path: str, name: str, data: np.ndarray, attr_dict=None, mode: str = "a"
) -> None:
    # Read h5 file
    hf = h5py.File(path, mode)
    # Create z_stack_dataset
    if hf.get(name) is None:
        data_shape = data.shape
        data_type = data.dtype
        max_shape = (data_shape[0],) + data_shape[1:]
        dset = hf.create_dataset(
            name,
            shape=data_shape,
            maxshape=max_shape,
            chunks=True,
            dtype=data_type,
            compression="gzip",
        )
        dset[:] = data
        if attr_dict is not None:
            for attr_key, attr_val in attr_dict.items():
                dset.attrs[attr_key] = attr_val
    else:
        print(f"Dataset {name} exists")

    hf.close()

def read_img(path):
    return skimage.io.imread(path, as_gray=True)

def joblib_loop(task, pics):
    return Parallel(n_jobs=20)(delayed(task)(i) for i in pics)

In [54]:
df_imgs_path = data_dir / 'basline' / 'metadata' / 'imgs.csv'
df_imgs_path.parent.mkdir(parents=True, exist_ok=True)

temp_path = data_dir  /'basline' /  'hdf5' / 'raw'
temp_path.mkdir(parents=True, exist_ok=True)

df_exist = df_imgs_path.is_file()


if not df_exist:
    print('Created df')
    
    group = df.groupby(['Condition', 'FOV'])
    rows = []

    for name, df_group in tqdm(group, total=len(group)):
        file_name = '_'.join(np.array(name).astype(str)) + '.hdf5'
        file_path = temp_path / file_name
        rows.append(list(name)+[file_path])
        
        group_cycle = df_group.groupby('Cycle')
        for cycle, df_cycle in group_cycle:
            channels = df_cycle.Channels.to_list()
            markers = df_cycle.Markers.to_list()
            paths = df_cycle.Path.to_numpy()
    
            imgs = joblib_loop(read_img, paths)
            imgs = np.array(imgs)
            info = {"Channels": channels, "Markers": markers}
            
            # hdf5 as Channel -> Z mapping
            save_hdf5(file_path, cycle, imgs, info)
    df_imgs = pd.DataFrame(rows, columns=['Condition', 'FOV', 'Path'])        
    df_imgs.to_csv(df_imgs_path, index=False)
else:
    print('Loaded df')
    df_imgs = pd.read_csv(df_imgs_path)

Created df


  0%|          | 0/14 [00:00<?, ?it/s]

In [55]:
df_cycle

,Condition,FOV,Cycle,Channels,Markers,Path
79,Treated,8,2,1,DNA,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."
80,Treated,8,2,3,Wnt1,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."
81,Treated,8,2,4,Tead,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."


# Save Tiffile

In [56]:
from skimage import exposure, util

def contrast_str(img, n_min=0.1, n_max=100):
    p2, p98 = np.percentile(img, (n_min, n_max))
    img_rescale = exposure.rescale_intensity(img, in_range=(p2, p98))
    return img_rescale

def make_imgs_same_dim(imgs):
    # Get max dimensions
    shapes = np.array([img.shape[1:] for img in imgs])
    min_x, min_y = shapes.min(axis=0)
    imgs_cropped = [img[:, :min_x, :min_y] for img in imgs]
    # imgs_cropped[0] = contrast_str(imgs_cropped[0])
    return imgs_cropped


In [57]:
save_path = data_dir /  'basline' /'imgs' / 'raw'
save_path.mkdir(parents=True, exist_ok=True)

group = df_imgs.groupby(['Condition', 'FOV'])
for name, df_group in group:
    path = df_group.iloc[0].Path
             
    # Read images
    cycles = []
    imgs_all = []
    channels = []
    with h5py.File(path, "r") as f:
        for k in tqdm(f.keys(), total=len(f.keys()), leave=False):
            cycle = k.split('_')[0]
            channel = f[k].attrs['Channels']

            imgs = f[k][:]
            cycles.append(cycle)
            channels.append(channel)
            imgs[0] = contrast_str(imgs[0])
            imgs_all.append(imgs)
    
    imgs_same_shape = make_imgs_same_dim(imgs_all)
    
    for i, imgs in enumerate(imgs_same_shape):
        temp_path = save_path / '_'.join(np.array(name).astype(str))
        temp_path.mkdir(parents=True, exist_ok=True)

        file_name = f'001_{cycles[i]}.tif'
        file_path = temp_path / file_name

        # Write image
        tiff.imwrite(file_path, imgs)


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

# Registration 

In [58]:
from ashlar import fileseries, thumbnail,reg
import matplotlib.pyplot as plt
from ashlar.scripts.ashlar import process_axis_flip

In [62]:
# Loop all images
imgs_dir = data_dir / 'basline' /'imgs' / 'raw'
save_dir = data_dir / 'basline' /'imgs' / 'registered'
save_dir.mkdir(parents=True, exist_ok=True)

imgs_dir_list = os.listdir(imgs_dir)

for dir_path in tqdm(imgs_dir_list):
    
    # Create reader for each cycle
    readers = []
    for i in range(1, 3):
        reader = fileseries.FileSeriesReader(
            str(imgs_dir / dir_path),
            pattern='{series}_'+f'{i}.tif',
            overlap=0.29,
            width=1,
            height=1,
            layout='snake',
            direction='horizontal',
            pixel_size=0.18872, 
        )
        readers.append(reader)
    reader_1 = readers[0]
    
    # Run stitching
    aligner0 = reg.EdgeAligner(reader_1, channel=0, filter_sigma=2, verbose=False,)
    aligner0.run()
    
    # Generate merge image for 1 cycle
    # Parramter
    mosaic_args = {}
    mosaic_args['verbose'] = False

    mosaic = reg.Mosaic(
            aligner0,aligner0.mosaic_shape,**mosaic_args
        )
    writer_class = reg.TiffListWriter
    writer = writer_class(
            [mosaic], str(save_dir / (dir_path + '_cycle1_ch{channel}.ome.tif'))
    )
    writer.run()
    
    # Loop through rest of cycles
    aligners = list()
    aligners.append(aligner0)

    for j in range(1, 2):
        aligners.append(
            reg.LayerAligner(readers[j], aligners[0], channel=0, filter_sigma=2, verbose=False)
        )
        aligners[j].run()
        mosaic = reg.Mosaic(
            aligners[j], aligners[0].mosaic_shape,**mosaic_args
        )
        writer = writer_class(
                [mosaic], str(save_dir / (dir_path +'_cycle'+str(j+1)+'_ch{channel}.ome.tif'))
        )
        writer.run()
    

  0%|          | 0/14 [00:00<?, ?it/s]

    assembling thumbnail 1/1
    assembling thumbnail 1/1
    estimated cycle offset [y x] = [-1.  2.]
    assembling thumbnail 1/1
    assembling thumbnail 1/1
    estimated cycle offset [y x] = [4. 1.]
    assembling thumbnail 1/1
    assembling thumbnail 1/1
    estimated cycle offset [y x] = [20.  6.]
    assembling thumbnail 1/1
    assembling thumbnail 1/1
    estimated cycle offset [y x] = [14.  0.]
    assembling thumbnail 1/1
    assembling thumbnail 1/1
    estimated cycle offset [y x] = [11. 14.]
    assembling thumbnail 1/1
    assembling thumbnail 1/1
    estimated cycle offset [y x] = [14.  3.]
    assembling thumbnail 1/1
    assembling thumbnail 1/1
    estimated cycle offset [y x] = [ 0. -1.]
    assembling thumbnail 1/1
    assembling thumbnail 1/1
    estimated cycle offset [y x] = [16.  0.]
    assembling thumbnail 1/1
    assembling thumbnail 1/1
    estimated cycle offset [y x] = [  0. -16.]
    assembling thumbnail 1/1
    assembling thumbnail 1/1
    estimated c

# Save data

In [4]:
def get_info(data_raw, marker_dict):
    conditions = []
    fovs = []
    cycles = []
    channels = []
    markers = []
    paths = [] 

    # Loop through image folder
    for (dirpath, dirnames, filenames) in os.walk(data_raw):
        for name in sorted(filenames):
            if "tif" in name:
                # Get information from image name
                n_split = name.split('_')
                                
                cond=n_split[0]
                fov=n_split[1]
                cycle=n_split[2]
                ch = n_split[3][:3]
                try:
                    marker = marker_dict[fov][cycle][ch]
                except:
                    continue 
                    
                conditions.append(cond)
                fovs.append(fov)
                cycles.append(cycle)
                channels.append(ch)
                markers.append(marker)
                paths.append(os.path.join(dirpath, name))
                
    info = {
            "Condition": conditions,
            "FOV": fovs,
            "Cycle": cycles,
            "Channels": channels,
            "Markers": markers,
            "Path": paths
        }

    df = pd.DataFrame(info)
    return df

def joblib_loop(task, pics):
    return Parallel(n_jobs=20)(delayed(task)(i) for i in pics)

def read_img(path):
    return skimage.io.imread(path, as_gray=True)

def get_min(imgs):
    shapes = np.array([np.array(img.shape) for img in imgs])
    return np.min(shapes, axis=0)

In [5]:
# from collections import defaultdict

# markers_map_new = defaultdict(dict)
# for k in markers_map.keys():
#     markers_map_new[k]['cycle1'] = dict()
#     markers_map_new[k]['cycle2'] = dict()
#     if 'cycle' in k:
#         continue
#     for i, (ch, marker) in enumerate(markers_map[k].items()):
#         markers_map_new[k]['cycle1'][f'ch{i}'] = marker
#     if k != '8':
#         for i, (ch, marker) in enumerate(markers_map['cycle2'].items()):
#             markers_map_new[k]['cycle2'][f'ch{i}'] = marker
#     else:
#         for i, (ch, marker) in enumerate(markers_map['cycle2_sec'].items()):
#             markers_map_new[k]['cycle2'][f'ch{i}'] = marker        

In [6]:
data_raw = data_dir  /  'basline' /'imgs' / 'registered'
df_meta_path = data_dir /  'basline' / 'metadata' / 'info_sti.csv'

try:
    df_meta_path.parent.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    print("Folder is already there")

df_exist = df_meta_path.is_file()

if not df_exist:
    print('Created df')
    df = get_info(data_raw, markers_map_new)
    df.to_csv(df_meta_path, index=False)
else:
    print('Loaded df')
    df = pd.read_csv(df_meta_path)

Folder is already there
Loaded df


In [4]:
df_imgs_path = data_dir / 'basline' /'metadata' / 'imgs_reg.csv'

temp_path =data_dir /  'basline' /'hdf5' / 'registered'
try:
    temp_path.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    print("Folder is already there")

df_exist = df_imgs_path.is_file()

if not df_exist:
    print('Created df')
    group = df.groupby(['Condition','FOV'])
    rows = []

    for name, df_group in tqdm(group, total=len(group)):
        file_name = '_'.join(np.array(name).astype(str)) + '.hdf5'
        file_path = temp_path / file_name
        rows.append(list(name)+[file_path])
        
        # if file_path.exists():
        #     continue
        
        channels = df_group.Channels.to_list()
        cycles = df_group.Cycle.to_list()
        markers = df_group.Markers.to_list()
        paths = df_group.Path.to_numpy()
            
        imgs = joblib_loop(read_img, paths)
        min_shape = get_min(imgs)
        imgs_cropped = np.array([img[:min_shape[0], :min_shape[1]] for img in imgs])
        info = {"Cycle": cycles, "Channel": channels, "Marker": markers}
            
            # hdf5 as Channel -> Z mapping
        save_hdf5(file_path, 'imgs', imgs_cropped, info)
    df_imgs = pd.DataFrame(rows, columns=['Condition', 'FOV', 'Path'])        
    df_imgs.to_csv(df_imgs_path, index=False)
else:
    print('Loaded df')
    df_imgs = pd.read_csv(df_imgs_path)

Folder is already there
Loaded df


# Segmentation

In [5]:
from skimage.exposure import match_histograms, rescale_intensity
import cv2
from skimage import img_as_ubyte
from skimage import exposure, util, filters, restoration

def random_crop(image, NEW_IMG_HEIGHT, NEW_IMG_WIDTH):
    cropped_image = tf.image.random_crop(
      image, size=[3, NEW_IMG_HEIGHT, NEW_IMG_WIDTH])

    return cropped_image

def make_color_overlay(input_data):
    """Create a color overlay from 2 channel image data
    
    Args:
        input_data: stack of input images
    
    Returns:
        numpy.array: color-adjusted stack of overlays in RGB mode
    """
    RGB_data = np.zeros(input_data.shape[:3] + (3, ), dtype='float32')
    
    # rescale channels to aid plotting
    for img in range(input_data.shape[0]):
        for channel in range(input_data.shape[-1]):
            # get histogram for non-zero pixels
            percentiles = np.percentile(input_data[img, :, :, channel][input_data[img, :, :, channel] > 0],
                                            [0, 100])
            rescaled_intensity = rescale_intensity(input_data[img, :, :, channel],
                                                       in_range=(percentiles[0], percentiles[1]),
                                                       out_range='float32')
            RGB_data[img, :, :, channel + 1] = rescaled_intensity
        
    # create a blank array for red channel
    return RGB_data

def contrast_str(img, n_min=10, n_max=100):
    p2, p98 = np.percentile(img, (n_min, n_max))
    img_rescale = rescale_intensity(img, in_range=(p2, p98))
    img_rescale = util.img_as_ubyte(img_rescale)
    return img_rescale

In [11]:
for row in tqdm(df_imgs.iloc[4:].itertuples(), total=len(df_imgs)):
    path = row.Path
    
    # Read images
    with h5py.File(path, "r") as f:
        
        markers = f['imgs'].attrs['Marker']
        imgs = f['imgs'][:]
    break

  0%|          | 0/14 [00:00<?, ?it/s]

In [12]:
import napari 

napari.view_image(np.stack(imgs), name=markers, channel_axis=0, contrast_limits=[0,1])

Viewer(camera=Camera(center=(0.0, 1737.5, 2327.0), zoom=0.14726133472400008, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(1.0, 1.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0.0, 3476.0, 1.0), (0.0, 4655.0, 1.0)), current_step=(1737, 2327), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'DNA' at 0x207e30fe430>, <Image layer 'CycD1/CDK4' at 0x20777ed0940>, <Image layer 'DNA [1]' at 0x20777ef6970>, <Image layer 'Concanavalin A' at 0x20777f2c880>, <Image layer 'Phalloidin' at 0x20777f5d820>, <Image layer 'WGA' at 0x20777f8cf40>], help='use <2> for transform', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_over_canvas=False, mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_double_click_callbacks=[], mouse_wheel_callbacks=[<function dims_scro

In [8]:
df_imgs['FOV'] = df_imgs['FOV'].astype(str)

In [9]:
# Define markers to use for cytosolic segmentation
cyto_markers = ['Phalloidin', 'WGA']

# Define folder and create if don't exsit
whole_seg_path = data_dir / 'basline'/ 'imgs' / 'segmentation'
whole_seg_path.mkdir(parents=True, exist_ok=True)

# Save combined images
for row in df_imgs.itertuples():
    # Read image
    path = row.Path
    with h5py.File(path, "r") as f:
        imgs = f['imgs'][:]
        markers = f['imgs'].attrs['Marker']

    # Get dapi and cyto imgaes
    indices = np.isin(markers, cyto_markers)
    img_dapi = imgs[2]
    imgs_cyto = imgs[indices,:]
    
    # Contrast streching and combine to rgb image
    img_dapi = contrast_str(img_dapi, n_max=99.9)
        
    if row.FOV == '8':
        img_cyto = np.zeros(img_dapi.shape)
    else:
        imgs_cyto_scaled = [contrast_str(imgs_cyto[0], n_max=99.9), contrast_str(imgs_cyto[1], n_max=99.9)]
        img_cyto = np.max(np.array(imgs_cyto_scaled), axis=0)
    img_rgb = np.stack([np.zeros(img_dapi.shape),img_cyto, img_dapi], axis=0).astype(np.uint8)
    
    # Crop and save
    file_name = f'{"_".join(row[1:3])}.tif'
    file_path = whole_seg_path / file_name
    tiff.imwrite(file_path, img_rgb)



# Segmentation

In [13]:
from skimage import exposure, util
from cellpose import core, utils, io, models, metrics

use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')

def contrast_str(img, n_min=0.01, n_max=99.95):
    p2, p98 = np.percentile(img, (n_min, n_max))
    img_rescale = exposure.rescale_intensity(img, in_range=(p2, p98))
    img_rescale = util.img_as_ubyte(img_rescale)
    return img_rescale

>>> GPU activated? YES


In [14]:
whole_seg_path = data_dir / 'basline'/ 'imgs' / 'segmentation'
mask_path = data_dir  / 'basline'/ 'imgs' / 'masks'

mask_path.mkdir(parents=True, exist_ok=True)

In [15]:
# Cyto segmentaion
masks = []
for p in os.listdir(whole_seg_path):
    if 'tif' not in p:
        continue
    img = skimage.io.imread(whole_seg_path / p).transpose((2,0,1))
    
    # Cyto segmentation
    model = models.CellposeModel(gpu=True, model_type='cyto2')
    mask_cyto, flows, styles = model.eval(img, 
                                  channels=[2,3],
                                  diameter=300,
                                  flow_threshold=0.8,
                                  cellprob_threshold=-3
                                  )
    
    file_path = mask_path / p
    tiff.imwrite(file_path, mask_cyto)
    
    # # Nuclei segemtnation
    # model = models.CellposeModel(gpu=True, model_type='nuclei')
    # mask_nuclei, flows, styles = model.eval(img, 
    #                               channels=[3,0],
    #                               diameter=100,
    #                               flow_threshold=0.6,
    #                               )
    
        
    # file_path = mask_path / f'Nuclei_{p}'
    # tiff.imwrite(file_path, mask_nuclei)


c:\Users\thu71\AppData\Local\anaconda3\envs\cellpose\lib\site-packages\cellpose\transforms.py:363: UserWarning: chan to seg' has value range of ZERO
  warnings.warn("chan to seg' has value range of ZERO")
